In [1]:
import pylab as plt
import tifffile as tif
from tifffile import imread
from tifffile import imsave
import PIL
from PIL import Image
import cv2
from itertools import product
import numpy as np
import pandas as pd
import os
import imagecodecs
import seaborn as sns
import matplotlib.cm as cm
import scipy as sp
from scipy import interpolate
from skimage import measure
from scipy import ndimage

"Ring Scan" functions

In [6]:
def dilate_mask(mask, it=8):
    #This function will take the pre-made binary mask and dilate it
    #Dilated masks have background signal at the periphery for reference
    kernel_size = 5  # Determines the thickness of the padding
    kernel = np.ones((kernel_size, kernel_size), np.uint8)

    padded_mask = cv2.dilate(mask, kernel, iterations=it)#2 iterations for preEx, 8 for postEx
    return padded_mask

def find_edge_pixels(mask):
    # Use OpenCV's findContours to find the edge pixels of the nucleus mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    edge_pixels = [tuple(point[0]) for point in contours[0]]
    return edge_pixels

def get_adjacent_pixels(pixel, shape):
    # Get the 8-neighboring pixels of a given pixel
    #pixel is the coordinates of the pixel of interest,
    #shape is the shape of the full image
    x, y = pixel
    neighbors = []
    for dx, dy in product([-1, 0, 1], repeat=2):
        nx, ny = x + dx, y + dy
        if 0 <= nx < shape[0] and 0 <= ny < shape[1]:
            neighbors.append((nx, ny))
    return neighbors

def create_ring_lists(image, mask, edge_pixels):
    #returns a list of lists
    #each element of the main list reresents a ring of pixels
        #starting with outer edge of the mask
    #each element of each sub-list is a set of pixel coordinates
    image=image.T
    mask=mask.T
    shape = image.shape
    visited = np.zeros(shape, dtype=bool)

    # Add the edge pixels to the visited array
    for pixel in edge_pixels:
        visited[pixel] = True

    ring_lists = [edge_pixels]
    count=0
    
    while True:
        ring = []
#         print(count)
        # Keep track of whether any unvisited pixels were found in this iteration
        found_unvisited = False

        for pixel in ring_lists[-1]:
            neighbors = get_adjacent_pixels(pixel, shape)
            
            for neighbor in neighbors:
                if not visited[neighbor] and neighbor not in ring_lists[-1] and mask[neighbor]!=0:
                    ring.append(neighbor)
                    visited[neighbor] = True
                    found_unvisited = True
        
        if not found_unvisited:
            break
        
        ring_lists.append(ring)
        count+=1
#         print(len(ring))

    return ring_lists

def get_mean_ringvals(image,ring_lists,normalized=True):
    
    image=image.T
    means=[np.mean([image[pixel[0], pixel[1]] for pixel in ring]) for ring in ring_lists]
#     peak=medians.index(max(medians))
#     nuc_interior=medians[peak:]
#     minval=min(nuc_interior)
#     maxval=medians[peak]
    if normalized==True:
        minval=min(means)
        maxval=max(means)
        means=[(x-minval) / (maxval-minval) for x in means]
    
    x_known=np.linspace(0,1,len(means))
    x_interp=np.linspace(0,1,500)
    means_interpolated=np.interp(x_interp,x_known,means)
    return means_interpolated

def imtoringvals(mask,image,it):
    mask=dilate_mask(mask,it=it)
    eps=find_edge_pixels(mask)
    rls=create_ring_lists(image,mask,eps)
    return get_mean_ringvals(image,rls,normalized=True)

def imtogradmap(image):
# Apply the Sobel operator for edge detection
    gradient_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=1)
    gradient_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=1)

    # Compute the gradient magnitude
    gradient_magnitude = np.sqrt(gradient_x**2 + gradient_y**2)
#     gradient_magnitude = cv2.GaussianBlur(gradient_magnitude,(5,5),0)
    return gradient_magnitude

Alternative linescan functions (Figure S8)

In [ ]:
def find_centroid(mask):
    # Calculating moments of the binary image
    M = cv2.moments(mask)

    # Calculating the centroid
    if M["m00"] != 0:
        centroid_x = int(M["m10"] / M["m00"])
        centroid_y = int(M["m01"] / M["m00"])
        return (centroid_x, centroid_y)
    else:
        return None

def generate_lines_and_values_with_mask(image, centroid, mask):
    height, width = image.shape
    lines_positive = [[] for _ in range(360)]
    lines_negative = [[] for _ in range(360)]

    for angle in range(360):
        rad = np.radians(angle)
        cos, sin = np.cos(rad), np.sin(rad)

        # Start from centroid and iterate through the image
        x, y = centroid
        while 0 <= x < width and 0 <= y < height and mask[int(y)][int(x)] == 1:
            lines_positive[angle].append(image[int(y)][int(x)])
            x += cos
            y += sin
        x_known=np.linspace(0,1,len(lines_positive[angle]))
        x_interp=np.linspace(0,1,500)
        lines_positive[angle] = np.interp(x_interp,x_known,lines_positive[angle])
        # Reset to centroid and iterate in the opposite direction
        x, y = centroid
        while 0 <= x < width and 0 <= y < height and mask[int(y)][int(x)] == 1:
            lines_negative[angle].append(image[int(y)][int(x)])  # Prepend to keep order from centroid outwards
            x -= cos
            y -= sin
        x_known=np.linspace(0,1,len(lines_negative[angle]))
        x_interp=np.linspace(0,1,500)
        lines_negative[angle] = np.interp(x_interp,x_known,lines_negative[angle])
    scans=[[sum(values) / len(values) for values in zip(*lines_positive)],[sum(values) / len(values) for values in zip(*lines_negative)]]
    meanscan=list([sum(values) / len(values) for values in zip(*scans)])
    minval=min(meanscan)
    maxval=max(meanscan)
    meanscan_norm=[(x-minval) / (maxval-minval) for x in meanscan]
    return list(reversed(meanscan_norm))

Derivative Estimation

In [ ]:
def numerical_derivative(y_values, samplerange=500):
    #Take interpolated mean pixel values from imtoringvals (y_values)
    #Returns the numerical derivative estimated within a given range (samplerange)
    # Number of points
    n = samplerange
    
    # x values are just the indices
    indices = np.arange(n)
    x_values =[i/n for i in indices]
    # Initialize an array to store the derivatives
    derivatives = np.zeros(n)

    # Compute derivatives using central difference method
    for i in range(1, n - 1):
        derivatives[i] = (y_values[i + 1] - y_values[i - 1]) / (x_values[i + 1] - x_values[i - 1])

    # For the first and last points, use forward and backward difference method
    derivatives[0] = (y_values[1] - y_values[0]) / (x_values[1] - x_values[0])
    derivatives[-1] = (y_values[-1] - y_values[-2]) / (x_values[-1] - x_values[-2])

    return derivatives